# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df_qty= data.groupby(['CustomerID', 'ProductName'])['Quantity'].sum().reset_index()

df_qty


,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
df_qty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63628 entries, 0 to 63627
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   63628 non-null  int64 
 1   ProductName  63628 non-null  object
 2   Quantity     63628 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [6]:
pivot_table= pd.pivot_table(df_qty,index='ProductName',columns= 'CustomerID', values='Quantity',fill_value=0)
pivot_table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [22]:
#Create all the metrics for euclidean (frame reference)


dist_pdist= pdist(X=pivot_table, metric='euclidean')
print(f'pdist calculation: {dist_pdist}')


dist_squareform= squareform(dist_pdist)
print(f'squareform calculation: {dist_squareform}')

euclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(pivot_table.T, 'euclidean'))),
                                index=pivot_table.columns,
                                columns=pivot_table.columns)

euclid_dist_norm


pdist calculation: [279.99642855 253.66907577 238.5120542  ... 264.14200726 268.28343221
 273.15014186]
squareform calculation: [[  0.         279.99642855 253.66907577 ... 255.67166444 258.0910692
  278.47800631]
 [279.99642855   0.         271.49953959 ... 265.7367118  279.55500353
  286.80655502]
 [253.66907577 271.49953959   0.         ... 245.27535547 253.01580978
  269.43644891]
 ...
 [255.67166444 265.7367118  245.27535547 ...   0.         264.14200726
  268.28343221]
 [258.0910692  279.55500353 253.01580978 ... 264.14200726   0.
  273.15014186]
 [278.47800631 286.80655502 269.43644891 ... 268.28343221 273.15014186
    0.        ]]


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [44]:
#Similiar customers for CustomerID 33

top_5 = list(euclid_dist_norm.sort_values(by=33,ascending=False))[:5]
top_5


[33, 200, 264, 356, 412]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [45]:
selected_records = df_qty.loc[top_5]
selected_records

,CustomerID,ProductName,Quantity
33,33,Phyllo Dough,1
200,356,Cake - Mini Cheesecake,1
264,412,Cheese - Camembert,1
356,464,Olive - Spread Tapenade,1
412,477,Isomalt,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [47]:
aggregated = selected_records.groupby('ProductName')['Quantity'].sum()

ranked = aggregated.sort_values(ascending=False)
print(ranked)

ProductName
Cake - Mini Cheesecake     1
Cheese - Camembert         1
Isomalt                    1
Olive - Spread Tapenade    1
Phyllo Dough               1
Name: Quantity, dtype: int64


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [55]:
merged_df = pd.merge(ranked,pivot_table, on='ProductName', how='inner')

purchased_products = selected_records['ProductName'].unique()

filtered_df = merged_df[~merged_df.isin(purchased_products)]
filtered_df.head(5)

,Quantity,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Cake - Mini Cheesecake,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Cheese - Camembert,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0,0.0,25.0
Isomalt,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Olive - Spread Tapenade,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Phyllo Dough,1,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [96]:
euclid_dist_norm = euclid_dist_norm.loc[df_qty['CustomerID'].unique()]

all_recommendations = {}

customer_ids = euclid_dist_norm.index.tolist()

# Iterate over it
for customer_id in customer_ids:
    top_5_similar_customers = euclid_dist_norm.loc[customer_id].sort_values(ascending=False)[:5].index.tolist()
    selected_records = df_qty[df_qty['CustomerID'].isin(top_5_similar_customers)]
    aggregated_rank = selected_records.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
    customer_purchased_products = df_qty[df_qty['CustomerID'] == customer_id]['ProductName'].tolist()
    filtered_df = aggregated_rank[~aggregated_rank.index.isin(customer_purchased_products)].head(5)    
    all_recommendations[customer_id] = filtered_df

for customer_id, recommendations in all_recommendations.items():
    print(f"Recommendations for CustomerID {customer_id}:")
    print(recommendations)
    print()


Recommendations for CustomerID 33:
ProductName
Butter - Unsalted               3
Soup - Campbells Bean Medley    3
Tart Shells - Sweet, 4          2
Scallops 60/80 Iqf              2
Garbage Bags - Clear            2
Name: Quantity, dtype: int64

Recommendations for CustomerID 200:
ProductName
Soup - Campbells Bean Medley       3
Pork - Kidney                      3
Muffin - Carrot Individual Wrap    3
Soupfoamcont12oz 112con            2
Ice Cream Bar - Hageen Daz To      2
Name: Quantity, dtype: int64

Recommendations for CustomerID 264:
ProductName
Bread - Italian Roll With Herbs    3
Wine - Two Oceans Cabernet         3
Veal - Inside, Choice              3
Durian Fruit                       2
Sprouts - Baby Pea Tendrils        2
Name: Quantity, dtype: int64

Recommendations for CustomerID 356:
ProductName
Veal - Inside, Choice        4
Butter - Unsalted            3
Tea - Herbal Sweet Dreams    2
Tart Shells - Sweet, 4       2
Foam Cup 6 Oz                2
Name: Quantity, dtype: i

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [105]:

rows = []

# Iterate over all_recommendations dictionary
for customer_id, recommendation in all_recommendations.items():
    # Extract the top 5 recommendations for the customer
    top_recommendations = recommendation.tolist()
    # Append the recommendations as a dictionary to the list of rows
    rows.append({'CustomerID': customer_id, 
                 'Recommendation1': top_recommendations[0],
                 'Recommendation2': top_recommendations[1],
                 'Recommendation3': top_recommendations[2],
                 'Recommendation4': top_recommendations[3],
                 'Recommendation5': top_recommendations[4]})

# Create DataFrame from the list of rows
recommendations_df = pd.DataFrame(rows)

# Display the DataFrame
recommendations_df


,CustomerID,Recommendation1,Recommendation2,Recommendation3,Recommendation4,Recommendation5
0,33,3,3,2,2,2
1,200,3,3,3,2,2
2,264,3,3,3,2,2
3,356,4,3,2,2,2
4,412,3,3,3,2,2
...,...,...,...,...,...,...
995,97928,27,25,22,22,21
996,98069,25,23,22,21,19
997,98159,26,25,25,24,20
998,98185,27,27,23,23,23


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.